In [25]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
import dill
from tqdm import tqdm 
from scipy.signal import find_peaks
import sys
import pyarrow as pa
import io

'''

'''

'\n\n'

In [26]:
sys.path.insert(0,'D:\OneDrive - University of Delaware - o365\Desktop\MatchPennies-Agent-Expirement')
import filter_functions as ff
from matplotlib.pyplot import cm

In [27]:
# Fields pull and pull list
os.chdir('D:\OneDrive - University of Delaware - o365\Subject_Data\MatchPennies_Agent_Exp2')
PATH = os.getcwd()

analysis_pull_list = []
with open("Analysis_Pull_List.txt", "r") as file:
    analysis_pull_list = file.read().splitlines()
analysis_pull_list_reaction = [s+'_Reaction' for s in analysis_pull_list]
analysis_pull_list_timing = [s+'_Timing' for s in analysis_pull_list]
analysis_pull_list_task = [s+'_Task' for s in analysis_pull_list]

with open("Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
    
num_subjects = len(analysis_pull_list)
task_name = 'Matchpennies_Agent_Exp2'

Target information


- Row 0 = Start Target

- Row 1 = Left screen, right target x pos (left target is 2\*startx - right targetx)

- Dim 1 = Radius

- Dim 2 = Thickness of the circle edge (don't know if this matters)

Dataframe is in centimeters, so need to divide everything by 100


In [28]:
file = 'Sub1_Reaction\\Sub1_ReactionTarget_Table.csv'
df = pd.read_csv(file)
df["X"] = df["X"]/100
df["Y"] = df["Y"]/100
df['Dim 1'] = df['Dim 1']/100 # Target table is in centimeters, I guess this doesn't matter but it makes me feel better
df['Dim 2'] = df['Dim 2']/100
# Target information for Right Hand (keeping this because the positions of target 3 and 4 are based on target 1 and start 1)
startx = df.loc[0]['X']
starty = df.loc[0]['Y']
start_radius = df.loc[0]['Dim 1'] 
adjusted_start_radius = start_radius*1.0 # ! This determines what size target I use to determine target leave time 

target1x = df.loc[1]['X']
target1y = df.loc[1]['Y']
target1_radius = df.loc[1]['Dim 1']

target2x = 2*startx - target1x
target2y = target1y
target2_radius = target1_radius

# Timing target
timing_targetx = startx
timing_targety = target1y
timing_target_pos = np.sqrt(timing_targetx**2 + timing_targety**2)
timing_target_radius = target1_radius   

# Parse Reaction Task Data


In [29]:
def get_force_reaction_time_single_trial(data):
    max_force = np.nanmax(data)
    index_max_force = np.nanargmax(data)
    max25 = 0.25*max_force
    max75 = 0.75*max_force
    max25_timepoint,_ = min(enumerate(data[:index_max_force]), key=lambda x: abs(x[1]-max25)) # Enumerate force to get timepoint and value, only slice up to the max value, then find where it's closest to 25percent of max value
    max75_timepoint,_ = min(enumerate(data[:index_max_force]), key=lambda x: abs(x[1]-max75))
    x1vals = max25_timepoint
    x2vals = max75_timepoint
    y1vals = max25
    y2vals = max75
    if x1vals - x2vals == 0:
        return np.nan
    slopes = (y2vals - y1vals)/(x2vals - x1vals)
    intercepts = y2vals - slopes*x2vals
    time_at_zero = -intercepts/slopes
    player_reaction_time_force = time_at_zero
    return player_reaction_time_force

In [30]:
#%% Get reaction time data
block_table = pd.read_csv(f'Sub1_Reaction\\Sub1_ReactionBlock_Table.csv')
num_trials = block_table['List_Reps'].loc[0]
num_blocks = sum(~block_table['TP_LIST'].isna())
trial_time = 8000
tot_trials = int(sum(block_table['List_Reps']))

filenames = np.empty((num_subjects, num_blocks,num_trials),dtype = object)
reaction_trial_start = np.zeros((num_subjects, num_blocks,num_trials))*np.nan
agent_reaction_leave_time = np.zeros((num_subjects, num_blocks,num_trials))*np.nan
agent_reaction_decision_array = np.empty((num_subjects, num_blocks,num_trials))*np.nan

reaction_trial_type_array = np.zeros((num_subjects, num_blocks,num_trials))*np.nan

reaction_xypos_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
reaction_dist_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan
reaction_xyvelocity_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
reaction_speed_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan
reaction_xyforce_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
reaction_force_data = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan


###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    subname = analysis_pull_list_reaction[i]
    print(i, subname)
    trial_table = pd.read_csv(subname+f'\\{subname}Trial_Table.csv')
    for x in range(tot_trials):
        block_trial_num = trial_table.iloc[x]['Block_Step']
        block_number = trial_table.iloc[x]['Block_Row']
        tp_num = trial_table.iloc[x]['TP_Row']
        j = tp_num - 1 # Block number
        k = block_trial_num - 1 # Trial number in that block
        
        filename = f"{subname}\\{task_name}_{subname}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        filenames[i,j,k] = filename # Store the file name
        reaction_df = pd.read_csv(filename,engine='c',dtype={'Event_Codes': 'object'})  # Read in data
        #* Find the trial start time with event codes
        event_code_col = reaction_df['Event_Codes']
        if 'E_GO_TRIAL' in event_code_col.unique():
            start_time = int(reaction_df[reaction_df['Event_Codes']=='E_GO_TRIAL'].index[-1]) #! Indexed at -1 bc if they need to redo the trial, we should use the last E_GO_TRIAL, not the first 
            reaction_trial_type_array[i,j,k] = 1 
        elif 'E_SHUTOFF_TRIAL' in event_code_col.unique():
            start_time = int(reaction_df[reaction_df['Event_Codes']=='E_SHUTOFF_TRIAL'].index[-1])
            reaction_trial_type_array[i,j,k] = 0
        else:
            raise Exception('ERROR, event code not found')
            
        #* Get Reaction Time Data
        end_time = start_time + trial_time      # Find how long the trial is, constant in this case to give people time to make it
        reaction_trial_start[i,j,k] = start_time         # Store the start times, NOT THE AGENT DECISION TIMES
        agent_reaction_leave_time[i,j,k] = reaction_df.iloc[start_time]['Agent_Decision_Time'] # Store the agent go time
        agent_reaction_decision_array[i,j,k] = reaction_df.iloc[start_time+1]['Agent_Target_Selection']

        cutoff_data = reaction_df.iloc[start_time:end_time] # Constrain data to the time that the trial starts, to the time that it ends
            
        reaction_xypos_data[i,j,k,:len(cutoff_data),:] = np.array(cutoff_data.drop(['Event_Codes'],axis=1)[['Left_HandX','Left_HandY']]) # Store left and right, slicing 3rd axis so if the trial ends early, then we just have nans at the end
        lhx = cutoff_data['Left_HandX'].to_numpy() # Left hand x position
        lhy = cutoff_data['Left_HandY'].to_numpy() # Left hand y position 
        lhx_vel = cutoff_data['Left_HandXVel'].to_numpy() # Left hand x velocity
        lhy_vel = cutoff_data['Left_HandYVel'].to_numpy() # Left hand y velocity
        lhx_force = cutoff_data['Left_FS_ForceX'].to_numpy()
        lhy_force = cutoff_data['Left_FS_ForceY'].to_numpy()
        
        # Filter force
        fx_nan_mask = ~np.isnan(lhx_force)
        fy_nan_mask = ~np.isnan(lhy_force)
        lhx_force_filt = np.zeros(trial_time)*np.nan
        lhy_force_filt = np.zeros(trial_time)*np.nan
        lhx_force_filt[:np.count_nonzero(fx_nan_mask)] = ff.Filter_KIN(lhx_force[fx_nan_mask])
        lhy_force_filt[:np.count_nonzero(fy_nan_mask)] = ff.Filter_KIN(lhy_force[fy_nan_mask])
        # Filter velocity
        vx_nan_mask = ~np.isnan(lhx_vel)
        vy_nan_mask = ~np.isnan(lhy_vel)
        lhx_vel_filt = np.zeros(trial_time)*np.nan
        lhy_vel_filt = np.zeros(trial_time)*np.nan
        lhx_vel_filt[:np.count_nonzero(fx_nan_mask)] = ff.Filter_KIN(lhx_vel[fx_nan_mask])
        lhy_vel_filt[:np.count_nonzero(fy_nan_mask)] = ff.Filter_KIN(lhy_vel[fy_nan_mask])
        
        #* Get force, speed, dist and store them in arrays
        reaction_xypos_data[i,j,k,:len(lhx),0] = lhx
        reaction_xypos_data[i,j,k,:len(lhx),1] = lhy
        dist = np.sqrt((lhx-startx)**2 + (lhy-starty)**2) # Calculate dist
        reaction_dist_data[i,j,k,:len(dist)] = dist # Store dist
        
        left_target_dist = np.sqrt(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2)) # Calculate dist
        reaction_dist_data[i,j,k,:len(dist)] = dist # Store dist
        dist = np.sqrt((lhx-startx)**2 + (lhy-starty)**2) # Calculate dist
        reaction_dist_data[i,j,k,:len(dist)] = dist # Store dist
        

        reaction_xyforce_data[i,j,k,:len(lhx_force_filt),0] = lhx_force_filt
        reaction_xyforce_data[i,j,k,:len(lhx_force_filt),1] = lhy_force_filt
        force = np.sqrt(lhx_force_filt**2 + lhy_force_filt**2)
        reaction_force_data[i,j,k,:] = force

        reaction_xyvelocity_data[i,j,k,:len(lhx_vel_filt),0] = lhx_vel_filt
        reaction_xyvelocity_data[i,j,k,:len(lhx_vel_filt),1] = lhy_vel_filt
        speed = np.sqrt((lhx_vel_filt)**2 + (lhy_vel_filt)**2) # Calculate velocity
        reaction_speed_data[i,j,k,:len(speed)] = speed # Store velocity 

 
 

0 Sub1_Reaction


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

Plot all the traces


# Pickle Reaction Stuff


In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = f'Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    print(data_path)
    dill.dump(filenames[i,:,:], open(data_path + f'{subname}_filenames.pkl','wb'))                     
    dill.dump(reaction_trial_start[i,:,:], open(data_path + f'{subname}_reaction_trial_start.pkl','wb'))                   
    dill.dump(reaction_trial_type_array[i,:,:], open(data_path + f'{subname}_reaction_trial_type_array.pkl','wb'))                   
    dill.dump(agent_reaction_leave_time[i,:,:], open(data_path + f'{subname}_agent_reaction_leave_time.pkl','wb'))  
    dill.dump(agent_reaction_decision_array[i,:,:], open(data_path + f'{subname}_agent_reaction_decision_array.pkl','wb')) 
    dill.dump(reaction_trial_type_array[i,:,:], open(data_path + f'{subname}_trial_type_array.pkl','wb'))              
    dill.dump(reaction_xypos_data[i,:,:], open(data_path + f'{subname}_reaction_xypos_data.pkl','wb'))           
    dill.dump(reaction_dist_data[i,:,:], open(data_path + f'{subname}_reaction_dist_data.pkl','wb'))            
    dill.dump(reaction_xyvelocity_data[i,:,:], open(data_path + f'{subname}_reaction_xyvelocity_data.pkl','wb'))      
    dill.dump(reaction_speed_data[i,:,:], open(data_path + f'{subname}_reaction_speed_data.pkl','wb'))           
    dill.dump(reaction_xyforce_data[i,:,:], open(data_path + f'{subname}_reaction_xyforce_data.pkl','wb'))         
    dill.dump(reaction_force_data[i,:,:], open(data_path + f'{subname}_reaction_force_data.pkl','wb'))           

Sub1
Subjects_Analyzed\Sub1\
Sub2
Subjects_Analyzed\Sub2\
Sub4
Subjects_Analyzed\Sub4\
Sub5
Subjects_Analyzed\Sub5\
Sub6
Subjects_Analyzed\Sub6\
Sub7
Subjects_Analyzed\Sub7\
Sub8
Subjects_Analyzed\Sub8\
Sub9
Subjects_Analyzed\Sub9\
Sub10
Subjects_Analyzed\Sub10\
Sub11
Subjects_Analyzed\Sub11\
Sub12
Subjects_Analyzed\Sub12\


In [ ]:
# i=-1
# for subname in analysis_pull_list:
#     i+=1
#     data_path = f'Subjects_Analyzed\\{subname}\\'
#     if not os.path.exists(data_path):
#         os.makedirs(data_path)
#     print(subname)
#     print(data_path)
#     dill.dump(reaction_dist_data[i,:,:], open(data_path + f'{subname}_reaction_dist_data.pkl', 'wb'))
#     dill.dump(reaction_yforce_data[i,:,:], open(data_path + f'{subname}_reaction_yforce_data.pkl', 'wb'))
#     dill.dump(player_reaction_time[i,:,:], open(data_path + f'{subname}_player_reaction_time.pkl', 'wb'))
#     dill.dump(player_yforce_reaction_time[i,:,:], open(data_path + f'{subname}_player_yforce_reaction_time.pkl', 'wb'))
#     dill.dump(player_reaction_movement_time[i,:,:], open(data_path + f'{subname}_player_reaction_movement_time.pkl', 'wb'))
#     dill.dump(player_yforce_reaction_movement_time[i,:,:],open(data_path + f'{subname}_player_yforce_reaction_movement_time.pkl', 'wb'))
#     dill.dump(player_reaction_plus_movement_time[i,:,:], open(data_path + f'{subname}_player_reaction_plus_movement_time.pkl', 'wb'))
#     dill.dump(player_reaction_decision_array[i,:,:], open(data_path + f'{subname}_player_reaction_decision_array.pkl', 'wb'))
#     dill.dump(agent_reaction_decision_array[i,:,:], open(data_path + f'{subname}_agent_reaction_decision_array.pkl', 'wb'))
#     dill.dump(agent_reaction_leave_time[i,:,:], open(data_path + f'{subname}_agent_reaction_leave_time.pkl', 'wb'))
#     dill.dump(trial_type_array[i,:,:], open(data_path + f'{subname}_reaction_trial_type_array.pkl', 'wb'))
#     # dill.dump(agent_decision_time[i,:], open(data_path + f'{subname}_agent_decision_time.pkl', 'wb'))


# Parse Timing Data


In [ ]:
coincidence_trial_time = 2500
coincidence_trials = 50
coincidence_trial_start = np.zeros((num_subjects, coincidence_trials))*np.nan
coincidence_reach_time = np.zeros((num_subjects, coincidence_trials))*np.nan

interval_trial_time = 2500
interval_trials = 50
interval_trial_start = np.zeros((num_subjects, interval_trials))*np.nan
interval_reach_time =  np.zeros((num_subjects, interval_trials))*np.nan

control_trials = interval_trials + coincidence_trials
###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_timing[i]
    print(i, sub_name)
    file = f'{sub_name}\\{sub_name}Trial_Table.csv'
    control_trial_table = pd.read_csv(file)
    block_number = 1
    tp_num = 1
    for x in ((range(control_trials))):
        block_number = control_trial_table.iloc[x]['Block_Row']
        tp_num = control_trial_table.iloc[x]['TP_Row']
        block_trial_num = control_trial_table.iloc[x]['Block_Step']
        filename = f'{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv'
        j = tp_num - 1 # Block number
        k = block_trial_num - 1 # Trial number in that block
        data = pd.read_csv(filename, low_memory=False)
        # if 'Agent_Initial_Time' in data.columns:
        #     data = data.rename(columns = {'Agent_Inital_Time':'Agent_Decision_Time'}, inplace=True)
        # f = data['Agent_Decision_Time']
        start_time = int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        # Get Interval Timing Data
        if tp_num == 1:
            end_time = start_time + interval_trial_time
            interval_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                interval_reach_time[i,k] = q[0]
        # Get Coincidence Timing Data
        if tp_num == 2:
            end_time = start_time + coincidence_trial_time
            coincidence_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                coincidence_reach_time[i,k] = q[0] 
                
            # #plot path for Reactions
            if False:
                for i in range(1):
                    player_reaction_time[0,0] = 500
                    plot_end_time = int(start_time + player_reaction_time[i,k])
                    # plot_end_time = int(start_time + 500)
                    lhx_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                    lhy_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                    plt.figure(dpi=300)
                    circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                    circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                    startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                    fig, ax = plt.subplots()
                    ax.add_patch(circleR)
                    ax.add_patch(circleL)
                    ax.add_patch(startCirc)
                    plt.plot(lhx_new,lhy_new) 
                    # plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])
                    print(player_reaction_time[i,k])
                    # plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                    plt.show()

0 Sub1_Timing
1 Sub2_Timing
2 Sub4_Timing
3 Sub5_Timing
4 Sub6_Timing
5 Sub7_Timing
6 Sub8_Timing
7 Sub9_Timing
8 Sub10_Timing
9 Sub11_Timing
10 Sub12_Timing


# Pickle Timing Data


In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = f'Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    print(data_path)
    dill.dump(interval_trial_start[i,:], open(data_path + f'{subname}_interval_trial_start.pkl', 'wb'))
    dill.dump(interval_reach_time[i,:], open(data_path + f'{subname}_interval_reach_time.pkl', 'wb'))
    dill.dump(coincidence_trial_start[i,:], open(data_path + f'{subname}_coincidence_trial_start.pkl', 'wb'))
    dill.dump(coincidence_reach_time[i,:], open(data_path + f'{subname}_coincidence_reach_time.pkl', 'wb'))
    

Sub1
Subjects_Analyzed\Sub1\
Sub2
Subjects_Analyzed\Sub2\
Sub4
Subjects_Analyzed\Sub4\
Sub5
Subjects_Analyzed\Sub5\
Sub6
Subjects_Analyzed\Sub6\
Sub7
Subjects_Analyzed\Sub7\
Sub8
Subjects_Analyzed\Sub8\
Sub9
Subjects_Analyzed\Sub9\
Sub10
Subjects_Analyzed\Sub10\
Sub11
Subjects_Analyzed\Sub11\
Sub12
Subjects_Analyzed\Sub12\


# Parse Task Data


In [ ]:
#%% Get task time task_df
# Get trials, blocks, trial_time from trial_table
path1 = PATH+'\\'+'Sub1_Task'
task_df = pd.read_csv(path1+f'\\Sub1_TaskTrial_Table.csv')
task_df = task_df.loc[task_df['Condition type']==3] # Only get the task condition 
num_trials = int(task_df.iloc[-1]['Block_Step']) # number of trials in each block
num_blocks = int(task_df.iloc[-1]['Block_Row'])
tot_trials = int(num_trials*num_blocks)
trial_time = int(task_df.iloc[0]['Condition time']) + 500
task_df_columns = len(fields_pull)
# Need to be kept outside subject for-loop
task_trial_start          = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
task_filenames            = np.empty((num_subjects, num_blocks,num_trials),dtype = object)
agent_task_leave_time     = np.zeros((num_subjects, num_blocks,num_trials))*np.nan
agent_task_decision_array = np.empty((num_subjects, num_blocks,num_trials))*np.nan
task_xypos_data           = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
task_dist_data            = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan
task_xyvelocity_data      = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
task_speed_data           = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan
task_xyforce_data         = np.zeros((num_subjects, num_blocks,num_trials, trial_time,2))*np.nan
task_force_data           = np.zeros((num_subjects, num_blocks,num_trials, trial_time))*np.nan

###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_task[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    trial_table = pd.read_csv(file)
    # Splt trial table into task and washout based on condition type
    if sub_name == 'Sub12_Task':
        task_trial_table = trial_table[trial_table['Condition_type']==3]
    else:
        task_trial_table = trial_table[trial_table['Condition type']==3]
    
    block_number = 1
    tp_num = 1
    for x in tqdm((range(tot_trials))):
        block_number = task_trial_table.iloc[x]['Block_Row']
        tp_num = task_trial_table.iloc[x]['TP_Row']
        block_trial_num = task_trial_table.iloc[x]['Block_Step']
        j = tp_num - task_trial_table['TP_Row'].min()  # Block number
        k = block_trial_num - 1 # Trial number in that block
        
        filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        task_filenames[i,j,k] = filename
        task_df = pd.read_csv(filename, low_memory=False)
        start_time = int(task_df[task_df['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        task_trial_start[i,j,k] = start_time
        end_time = start_time + trial_time
        task_df = task_df.drop(columns ='Event_Codes') # Drop event codes cuz it's not number column and can't be an array 
        cutoff_data = task_df.iloc[start_time:end_time]
        agent_task_decision_array[i,j,k] = task_df.iloc[start_time+1]['Agent_Target_Selection']
        agent_task_leave_time[i,j,k] = task_df.iloc[start_time]['Agent_Decision_Time'] # Store the agent go time

        
        lhx = cutoff_data['Left_HandX'].to_numpy() # Left hand x position
        lhy = cutoff_data['Left_HandY'].to_numpy() # Left hand y position 
        lhx_vel = cutoff_data['Left_HandXVel'].to_numpy() # Left hand x velocity
        lhy_vel = cutoff_data['Left_HandYVel'].to_numpy() # Left hand y velocity
        lhx_force = cutoff_data['Left_FS_ForceX'].to_numpy()
        lhy_force = cutoff_data['Left_FS_ForceY'].to_numpy()
        
        # Filter force
        fx_nan_mask = ~np.isnan(lhx_force)
        fy_nan_mask = ~np.isnan(lhy_force)
        lhx_force_filt = np.zeros(trial_time)*np.nan
        lhy_force_filt = np.zeros(trial_time)*np.nan
        lhx_force_filt[:np.count_nonzero(fx_nan_mask)] = ff.Filter_KIN(lhx_force[fx_nan_mask])
        lhy_force_filt[:np.count_nonzero(fy_nan_mask)] = ff.Filter_KIN(lhy_force[fy_nan_mask])
        # Filter velocity
        vx_nan_mask = ~np.isnan(lhx_vel)
        vy_nan_mask = ~np.isnan(lhy_vel)
        lhx_vel_filt = np.zeros(trial_time)*np.nan
        lhy_vel_filt = np.zeros(trial_time)*np.nan
        lhx_vel_filt[:np.count_nonzero(fx_nan_mask)] = ff.Filter_KIN(lhx_vel[fx_nan_mask])
        lhy_vel_filt[:np.count_nonzero(fy_nan_mask)] = ff.Filter_KIN(lhy_vel[fy_nan_mask])
        
        #* Get force, speed, dist and store them in arrays
        task_xypos_data[i,j,k,:len(lhx),0] = lhx
        task_xypos_data[i,j,k,:len(lhx),1] = lhy
        dist = np.sqrt((lhx-startx)**2 + (lhy-starty)**2) # Calculate dist
        task_dist_data[i,j,k,:len(dist)] = dist # Store dist
        
        left_target_dist = np.sqrt(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2)) # Calculate dist
        task_dist_data[i,j,k,:len(dist)] = dist # Store dist
        dist = np.sqrt((lhx-startx)**2 + (lhy-starty)**2) # Calculate dist
        task_dist_data[i,j,k,:len(dist)] = dist # Store dist
        

        task_xyforce_data[i,j,k,:len(lhx_force_filt),0] = lhx_force_filt
        task_xyforce_data[i,j,k,:len(lhx_force_filt),1] = lhy_force_filt
        force = np.sqrt(lhx_force_filt**2 + lhy_force_filt**2)
        task_force_data[i,j,k,:] = force

        task_xyvelocity_data[i,j,k,:len(lhx_vel_filt),0] = lhx_vel_filt
        task_xyvelocity_data[i,j,k,:len(lhx_vel_filt),1] = lhy_vel_filt
        speed = np.sqrt((lhx_vel_filt)**2 + (lhy_vel_filt)**2) # Calculate velocity
        task_speed_data[i,j,k,:len(speed)] = speed # Store velocity 
           

0 Sub1_Task


100%|██████████| 320/320 [00:18<00:00, 17.28it/s]


1 Sub2_Task


100%|██████████| 320/320 [00:10<00:00, 30.59it/s]


2 Sub4_Task


...

100%|██████████| 320/320 [00:08<00:00, 38.75it/s]


9 Sub11_Task


100%|██████████| 320/320 [00:11<00:00, 26.91it/s]


10 Sub12_Task


100%|██████████| 320/320 [00:08<00:00, 38.10it/s]


# Pickle Task Data


In [ ]:
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    dill.dump(task_trial_start[i,...], open(data_path + f'{subname}_task_trial_start.pkl','wb'))          
    dill.dump(task_filenames[i,...], open(data_path + f'{subname}_task_filenames.pkl','wb'))            
    dill.dump(agent_task_leave_time[i,...], open(data_path + f'{subname}_agent_task_leave_time.pkl','wb'))  
    dill.dump(agent_task_decision_array[i,...], open(data_path + f'{subname}_agent_task_decision_array.pkl','wb')) 
    dill.dump(task_xypos_data[i,...], open(data_path + f'{subname}_task_xypos_data.pkl','wb'))           
    dill.dump(task_dist_data[i,...], open(data_path + f'{subname}_task_dist_data.pkl','wb'))            
    dill.dump(task_xyvelocity_data[i,...], open(data_path + f'{subname}_task_xyvelocity_data.pkl','wb'))      
    dill.dump(task_speed_data[i,...], open(data_path + f'{subname}_task_speed_data.pkl','wb'))           
    dill.dump(task_xyforce_data[i,...], open(data_path + f'{subname}_task_xyforce_data.pkl','wb'))         
    dill.dump(task_force_data[i,...], open(data_path + f'{subname}_task_force_data.pkl','wb')) 

Sub1
Sub2
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12
